# Preparation 

## Install Related Packages

Note that you have to restart the runtime after all the following installation.

In [ ]:
!pip install --upgrade transformers
!pip install --upgrade simpletransformers
!pip install --upgrade rouge-metric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.2 MB/s 
     |████████████████████████████████| 6.6 MB 41.5 MB/s 
     |████████████████████████████████| 120 kB 68.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 5.1 MB/s 
     |████████████████████████████████| 365 kB 60.2 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 1.3 MB 52.5 MB/s 
     |████████████████████████████████| 9.1 MB 48.4 MB/s 
     |████████████████████████████████| 1.8 MB 57.9 MB/s 
     |████████████████████████████████| 181 kB 64.8 MB/s 
     |████████████████████████████████| 157 kB 51.6 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 157 kB 70.0 MB/s 
     |████████████████████████████████| 157 kB 64.3 MB

## Mount Your Google Drive for Saving

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Download Summarization Dataset

In [ ]:
# !wget -O CLTS.zip https://www.dropbox.com/s/slrq9yaphxaax0v/CLTS.zip?dl=0
# !wget -O CLTS.zip https://www.dropbox.com/s/0pm3ql0ybjiok0f/CLTS_temp.zip?dl=0
!cp /content/drive/MyDrive/CLTS.zip /content/CLTS.zip
!unzip CLTS.zip
!rm CLTS.zip

Archive:  CLTS.zip
   creating: CLTS/
  inflating: __MACOSX/._CLTS         
  inflating: CLTS/train.tgt          
  inflating: __MACOSX/CLTS/._train.tgt  
  inflating: CLTS/test.src           
  inflating: __MACOSX/CLTS/._test.src  
  inflating: CLTS/.DS_Store          
  inflating: __MACOSX/CLTS/._.DS_Store  
  inflating: CLTS/valid.src          
  inflating: __MACOSX/CLTS/._valid.src  
  inflating: CLTS/test.tgt           
  inflating: __MACOSX/CLTS/._test.tgt  
  inflating: CLTS/train.src          
  inflating: __MACOSX/CLTS/._train.src  


## Import Packages

In [ ]:
import math
import os
import random
import time
import sys

from scipy import spatial
import numpy as np
from sklearn.utils import shuffle
import pandas as pd
from simpletransformers.language_representation import RepresentationModel
from simpletransformers.t5 import T5Model, T5Args
import torch

from rouge_metric import PyRouge
rouge = PyRouge()

# Deal with Summarization Dataset (CLTS)

In [ ]:
n = 0
with open("./CLTS/train.src") as fsrc, open("./CLTS/train.tgt") as ftgt:
    # [TODO] 使用使用zip()同時讀取 .src 和 .tgt 的資料（ex: 同時處理第一筆src和第一筆tgt，然後是第二筆src和第二筆tgt，...)
    ##################################################
    for src, tgt in zip(fsrc, ftgt):
    ##################################################
        n += 1
        print("No.", n)
        print("src:", src)
        print("tgt:", tgt)
        if n > 3:
            break

No. 1
src: 山 西 省 晋 中 市 祁 县 乔 家 大 院 景 区 内 。   IC   资 料 图 山 西 省 晋 中 市 乔 家 大 院 景 区 在 被 文 化 和 旅 游 部 取 消 旅 游 景 区 质 量 等 级 次 日 ， 晋 中 市 召 开 专 题 会 议 研 究 部 署 整 改 工 作 等 。 据 《 晋 中 日 报 》 报 道 ， 8 月 1 日 下 午 ， 晋 中 市 委 书 记 赵 建 平 主 持 召 开 专 题 会 议 ， 研 究 部 署 乔 家 大 院 景 区 再 创 5A 品 牌 整 改 提 升 工 作 。 他 强 调 ， 要 深 刻 汲 取 乔 家 大 院 景 区 摘 牌 教 训 ， 全 面 认 领 问 题 清 单 ， 以 铁 的 手 腕 、 铁 的 措 施 ， 从 严 从 实 彻 查 整 改 ， 以 脱 胎 换 骨 、 凤 凰 涅 槃 的 景 区 形 象 及 时 回 应 社 会 关 切 。 市 委 常 委 、 秘 书 长 、 统 战 部 长 鹿 建 平 ， 副 市 长 辛 琰 、 郝 向 明 参 加 。 在 认 真 听 取 相 关 整 改 提 升 工 作 情 况 汇 报 后 ， 赵 建 平 指 出 ， 国 家 文 化 和 旅 游 部 发 布 公 告 取 消 了 乔 家 大 院 5A 景 区 质 量 等 级 ， 反 响 强 烈 、 教 训 深 刻 。 要 主 动 开 展 自 我 剖 析 ， 详 细 制 订 整 改 方 案 ， 以 更 严 的 标 准 、 更 优 的 服 务 ， 认 真 落 实 整 改 措 施 ， 确 保 在 最 短 时 间 内 取 得 显 著 成 效 ， 重 新 达 到 5A 评 估 标 准 。 就 进 一 步 抓 好 乔 家 大 院 景 区 整 改 提 升 工 作 ， 赵 建 平 提 出 了 五 点 要 求 ， 具 体 包 括 ： 一 要 找 准 问 题 症 结 ， 对 照 国 家 文 化 和 旅 游 部 督 察 指 出 的 问 题 ， 拉 出 整 改 清 单 和 任 务 表 ， 逐 条 逐 项 细 化 整 改 措 施 ， 挂 图 作 战 、 倒 排 工 期 ， 逐 一 对 账 销 号 ， 确 保 问 题 全 部 整 改 到 位 ； 二 要 理 顺 体 制 机 制 ， 确 定 管 理 主 体 ， 划 定 政 府

In [ ]:
data_dir = "./CLTS" 
data_type = "train"
max_len = 20
print("PATH:", os.path.join(data_dir, "%s.src" % data_type))

with open(os.path.join(data_dir, "%s.src" % data_type)) as fsrc:
    for src in fsrc:
        # [TODO] 對src進行以下處理，並觀察結果
        # (1) 移除句子中的空白分隔 src = src.replace(...)
        # (2) 使句子長度最長為max_len，超過部分直接忽略 src = src[...]
        ##################################################
        src = src.replace(" ", "")[:max_len]
        ##################################################
        print(src)
        print("Length of src:", len(src))
        break


PATH: ./CLTS/train.src
山西省晋中市祁县乔家大院景区内。IC资料
Length of src: 20


# Creation of an mT5 Summarization Model




In [ ]:
class T5Summarizer:
    def __init__(self, max_seq_len=500, max_output_len=300, lang='zh'):
        self.max_seq_len = max_seq_len
        self.max_output_len = max_output_len
        self.lang = lang
        self.model = None

    def load_default_model(self, model_path="google/mt5-small"):
        self.model = T5Model("mt5", model_path, use_cuda=True)

    def create_model(self, epoch=1):
        self.model_args = T5Args()
        self.model_args.max_seq_length = self.max_seq_len
        self.model_args.max_length = self.max_output_len
        self.model_args.train_batch_size = 2
        self.model_args.eval_batch_size = 8
        self.model_args.num_train_epochs = epoch
        self.model_args.evaluate_during_training = False
        self.model_args.evaluate_during_training_steps = 300000
        self.model_args.use_multiprocessing = False
        self.model_args.fp16 = False
        self.model_args.save_steps = -1
        self.model_args.save_eval_checkpoints = False
        self.model_args.no_cache = True
        self.model_args.reprocess_input_data = True
        self.model_args.overwrite_output_dir = True
        self.model_args.num_return_sequences = 1
        self.model_args.output_dir = "outputs/"
        self.model = T5Model("mt5", "google/mt5-small", args=self.model_args, use_cuda=True)

    def load_data(self, data_type):
        data_dir = "./CLTS"
        # [TODO] 使用使用zip()同時讀取 .src 和 .tgt 的資料（ex: 同時處理第一筆src和第一筆tgt，然後是第二筆src和第二筆tgt，...)
        # 然後，對src和tgt進行以下處理
        # (1) 移除句子中的空白分隔
        # (2) 使句子長度最長為self.max_seq_len，超過部分直接忽略
        # 最後使用 yield (src, tgt) 回傳處理後的句子
        # with open ...
        #     for ...
        #         src = src.replace(...)[...]
        #         tgt = tgt.replace(...)[...]
        #         yield (src, tgt)
        ##################################################      
        with open(os.path.join(data_dir, "%s.src" % data_type)) as fsrc, open(os.path.join(data_dir, "%s.tgt" % data_type)) as ftgt:
            for src, tgt in zip(fsrc, ftgt):
                src = src.replace(" ", "")[:self.max_seq_len]
                tgt = tgt.replace(" ", "")[:self.max_seq_len]
                yield (src, tgt)
        ##################################################

    def train(self, train_size=None, epoch=1):
        torch.cuda.empty_cache()
        torch.cuda.memory_summary(device=None, abbreviated=False)
        self.create_model(epoch=1)
        train_data = []
        for src, tgt in self.load_data("train"):
            train_data.append({"input_text": src, "target_text": tgt})
            if train_size and len(train_data) >= train_size:
                break
        print("Number of training data: %d" % len(train_data))
        train_df = pd.DataFrame(train_data)
        train_df["prefix"] = "summarize"
        self.model.train_model(train_df)

    def evaluate(self, valid_size=None):
        eval_data = []
        references = []
        for src, tgt in self.load_data("test"):
            eval_data.append("summarize: %s" % src)
            if self.lang == 'zh':
                tgt = " ".join(list(tgt))
            references.append([tgt])
            if valid_size and len(eval_data) >= valid_size:
                break
        predictions = self.model.predict(eval_data)
        if self.lang == 'zh':
            predictions = [" ".join(list(p)) for p in predictions]
        print("Number of test data: %d" % len(predictions))
        return rouge.evaluate(predictions, references)

    def perform(self, text, headline="", target_len=None):
        if not self.model:
            self.load_default_model()
        if not target_len:
            target_len = self.max_output_len
        result = self.model.predict(["summarize: %s" % text])[0]
        return result[:target_len]


In [ ]:
model = T5Summarizer()

for src, tgt in model.load_data("test"):
    print(src)
    print(tgt)
    break

中新网3月4日电国台办发言人朱凤莲3月4日表示，由于民进党当局一再阻挠，1148名急需返乡的滞鄂台胞迄今无法回家。苏贞昌日前又公开散布“苏式谎言”，继续罔顾事实、颠倒黑白，谎称“卡关就卡在大陆”，“真不知人间还有羞耻二字。”朱凤莲说，疫情发生以来，大陆方面一方面全力照顾在大陆台胞的生活和疫情防控需要，另一方面充分考虑滞鄂台胞的实际需求和回家心愿，积极安排东航于2月3日运送首批247名台胞返回台湾，并于2月5日和此后多次提出尽快运送其他提出返乡要求台胞的合理安排，包括提出由两岸航空公司共同执飞临时航班的运送安排，以满足滞鄂台胞急切回家的愿望。但民进党当局却一而再、再而三变换借口，不断设置障碍，一再拖延阻挠。“2月15日，我办发言人已详细披露大陆方面持续做出运送台胞安排和为实现运送不懈努力的全过程和细节，具体情况清清楚楚，事实真相一目了然。”朱凤莲指出，民进党当局不断以各种借口阻止东航后续运送，有目共睹。苏贞昌自己就曾公开说过，不能让在湖北的台胞回去，是因为岛内防疫安置能量不足。更有甚者，民进党当局竟然将期待返乡就业、学习团聚等1148名台胞列入所谓“注记管制名单”，全面封堵了滞鄂台胞回家
朱凤莲表示，事实反复证明，民进党当局根本就不想让在湖北的台胞回家，滞鄂台胞返乡之路受阻，“卡关”就卡在民进党当局的这些政客手中。



### 完整版 model training & evaluation

In [ ]:
model.train()

Number of training data: 148317


  0%|          | 0/148317 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/74159 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

因為完整 data 筆數太多，無法在短時間內完成 training 和 evaluation (可以課後自行嘗試)，所以先提供訓練好的 model 以供測試 predict 的結果。

In [ ]:
#!wget -O CLTS_model.zip https://www.dropbox.com/s/8v37nn666c1kng7/CLTS_model.zip?dl=0
!cp /content/drive/MyDrive/CLTS_model.zip /content/CLTS_model.zip
!unzip CLTS_model.zip
!rm CLTS_model.zip

Archive:  CLTS_model.zip
   creating: CLTS_model/
  inflating: __MACOSX/._CLTS_model   
  inflating: CLTS_model/model_args.json  
  inflating: __MACOSX/CLTS_model/._model_args.json  
  inflating: CLTS_model/tokenizer_config.json  
  inflating: __MACOSX/CLTS_model/._tokenizer_config.json  
  inflating: CLTS_model/special_tokens_map.json  
  inflating: __MACOSX/CLTS_model/._special_tokens_map.json  
  inflating: CLTS_model/optimizer.pt  
  inflating: __MACOSX/CLTS_model/._optimizer.pt  
  inflating: CLTS_model/config.json  
  inflating: __MACOSX/CLTS_model/._config.json  
  inflating: CLTS_model/scheduler.pt  
  inflating: __MACOSX/CLTS_model/._scheduler.pt  
  inflating: CLTS_model/training_args.bin  
  inflating: __MACOSX/CLTS_model/._training_args.bin  
  inflating: CLTS_model/spiece.model  
  inflating: __MACOSX/CLTS_model/._spiece.model  
  inflating: CLTS_model/pytorch_model.bin  
  inflating: __MACOSX/CLTS_model/._pytorch_model.bin  


In [ ]:
model.load_default_model("./CLTS_model/")

In [ ]:
model.evaluate()

Generating outputs:   0%|          | 0/2086 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/16687 [00:00<?, ?it/s]

Number of test data: 16687


{'rouge-1': {'r': 0.4631826110800003,
  'p': 0.45015108799771014,
  'f': 0.45657388209770217},
 'rouge-2': {'r': 0.34447691943493075,
  'p': 0.3349685591771243,
  'f': 0.33965620790826406},
 'rouge-l': {'r': 0.4074163383475741,
  'p': 0.39614524675527873,
  'f': 0.4017017460240432}}

In [ ]:
src = "3 月 25 日 ， 杭 州 市 小 客 车 总 量 调 控 管 理 办 公 室 发 布 公 告 称 ， 根 据 省 、 市 党 委 政 府 关 于 加 快 提 振 汽 车 消 费 ， 促 进 经 济 发 展 相 关 工 作 的 要 求 ， 经 研 究 决 定 ， 2020 年 一 次 性 增 加 2 万 个 小 客 车 指 标 ， 均 为 摇 号 指 标 ， 将 在 今 年 上 半 年 配 置 完 毕 。 杭 州 市 小 客 车 总 量 调 控 管 理 办 公 室 发 布 的 《 关 于 2020 年 一 次 性 增 加 小 客 车 指 标 的 配 置 公 告 》 具 体 规 定 这 2 万 个 小 客 车 指 标 全 部 是 摇 号 指 标 。 其 中 ， 个 人 阶 梯 摇 号 15000 个 ， 将 于 2020 年 5 月 29 日 组 织 一 次 个 人 阶 梯 摇 号 ， 完 成 15000 个 指 标 的 配 置 ； 新 增 的 5000 个 县 （ 市 ） 指 标 将 在 2020 年 5 月 18 、 19 日 举 行 的 第 十 次 小 客 车 县 （ 市 ） 指 标 摇 号 活 动 中 与 原 定 指 标 配 额 合 并 投 放 。 分 配 至 各 县 （ 市 ） 的 指 标 比 例 具 体 为 建 德 市 1833 个 ， 淳 安 县 1667 个 ， 桐 庐 县 1500 个 。 据 悉 ， 杭 州 市 的 小 客 车 指 标 有 摇 号 和 竞 拍 两 种 配 置 方 式 ， 去 年 ， 全 年 杭 州 市 个 人 摇 号 指 标 达 58005 个 ， 此 次 新 增 的 2 万 个 指 标 占 比 大 约 为 34 . 5 % 。 而 杭 州 去 年 个 人 竞 价 的 指 标 数 量 为 14300 个 。 近 日 ， 浙 江 省 人 民 政 府 办 公 厅 发 布 《 关 于 提 振 消 费 促 进 经 济 稳 定 增 长 的 实 施 意 见 》 ， 其 中 一 条 是 要 释 放 城 乡 汽 车 消 费 潜 力 。 上 述 意 见 指 出 了 几 个 方 向 ， 第 一 条 便 是 鼓 励 杭 州 有 序 放 宽 汽 车 限 购 措 施 ， 制 定 汽 车 以 旧 换 新 和 下 乡 惠 农 政 策 ， 深 挖 农 村 汽 车 消 费 潜 力 。 值 得 注 意 的 是 ， 在 春 节 销 售 淡 季 叠 加 疫 情 影 响 ， 中 国 车 市 刚 刚 经 历 了 艰 难 的 开 局 。 据 乘 联 会 数 据 显 示 ， 2 月 我 国 狭 义 乘 用 车 市 场 产 量 和 批 发 销 量 分 别 为 21 . 5 万 辆 、 25 . 2 万 辆 ， 同 比 骤 减 80 . 6 % 、 78 . 5 % ， 销 量 规 模 相 当 于 2006 年 2 月 水 平 。 1 - 2 月 累 计 批 发 销 量 196 . 9 万 辆 ， 同 比 下 降 41 % ， 乘 联 会 也 再 次 调 低 了 全 年 的 销 量 预 测 增 长 率 到 - 8 % 。 在 此 背 景 之 下 ， 自 疫 情 爆 发 以 来 ， 国 家 层 面 已 多 次 发 文 鼓 励 汽 车 消 费 。 2 月 16 日 ， 《 求 是 》 杂 志 发 布 的 中 央 重 要 文 章 中 指 出 ， 要 稳 定 居 民 消 费 ， 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 配 额 。 2 月 20 日 ， 商 务 部 在 线 上 新 闻 发 布 会 中 ， 也 提 出 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 配 额 ， 带 动 汽 车 消 费 。 3 月 13 日 ， 国 家 发 改 委 、 商 务 部 等 23 个 部 门 向 各 省 市 联 合 发 布 《 关 于 促 进 消 费 扩 容 提 质   加 快 形 成 强 大 国 内 市 场 的 实 施 意 见 》 ， 提 出 要 促 进 汽 车 限 购 向 引 导 使 用 政 策 转 变 ， 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 限 额 。 而 在 刺 激 车 市 的 措 施 中 ， 放 开 部 分 城 市 的 汽 车 限 购 在 不 少 业 内 人 士 看 来 是 最 为 直 接 ， 同 时 也 是 风 险 相 对 较 低 的 做 法 。 乘 联 会 秘 书 长 崔 东 树 就 指 出 ， 如 果 限 购 城 市 被 限 制 的 50 万 - 60 万 个 牌 照 配 额 有 序 放 开 ， 可 能 会 带 来 20 万 - 30 万 辆 的 销 量 ， 对 提 振 车 市 、 促 进 消 费 信 心 恢 复 具 有 积 极 的 作 用 。 目 前 ， 全 国 实 行 限 购 的 省 市 有 北 京 、 上 海 、 广 州 、 石 家 庄 、 天 津 、 杭 州 、 深 圳 、 海 南 ， 这 些 城 市 居 民 购 车 要 先 通 过 摇 号 或 拍 牌 的 方 式 获 取 指 标 。 此 前 ， 有 消 息 传 出 ， 北 京 将 率 先 “ 松 绑 ” 汽 车 限 购 ， 包 括 释 放 10 万 辆 新 能 源 汽 车 指 标 在 内 的 多 项 措 施 。 但 在 3 月 25 日 ， 北 京 市 商 务 局 在 官 方 微 博 上 发 表 致 歉 声 明 称 ， 以 上 措 施 均 未 经 过 研 究 论 证 。 ( 本 文 来 自 澎 湃 新 闻 ， 更 多 原 创 资 讯 请 下 载 “ 澎 湃 新 闻 ” APP )".replace(" ", "")
prd = model.perform(src)
print("src: ", src)
print("prd:", prd)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

src:  3月25日，杭州市小客车总量调控管理办公室发布公告称，根据省、市党委政府关于加快提振汽车消费，促进经济发展相关工作的要求，经研究决定，2020年一次性增加2万个小客车指标，均为摇号指标，将在今年上半年配置完毕。杭州市小客车总量调控管理办公室发布的《关于2020年一次性增加小客车指标的配置公告》具体规定这2万个小客车指标全部是摇号指标。其中，个人阶梯摇号15000个，将于2020年5月29日组织一次个人阶梯摇号，完成15000个指标的配置；新增的5000个县（市）指标将在2020年5月18、19日举行的第十次小客车县（市）指标摇号活动中与原定指标配额合并投放。分配至各县（市）的指标比例具体为建德市1833个，淳安县1667个，桐庐县1500个。据悉，杭州市的小客车指标有摇号和竞拍两种配置方式，去年，全年杭州市个人摇号指标达58005个，此次新增的2万个指标占比大约为34.5%。而杭州去年个人竞价的指标数量为14300个。近日，浙江省人民政府办公厅发布《关于提振消费促进经济稳定增长的实施意见》，其中一条是要释放城乡汽车消费潜力。上述意见指出了几个方向，第一条便是鼓励杭州有序放宽汽车限购措施，制定汽车以旧换新和下乡惠农政策，深挖农村汽车消费潜力。值得注意的是，在春节销售淡季叠加疫情影响，中国车市刚刚经历了艰难的开局。据乘联会数据显示，2月我国狭义乘用车市场产量和批发销量分别为21.5万辆、25.2万辆，同比骤减80.6%、78.5%，销量规模相当于2006年2月水平。1-2月累计批发销量196.9万辆，同比下降41%，乘联会也再次调低了全年的销量预测增长率到-8%。在此背景之下，自疫情爆发以来，国家层面已多次发文鼓励汽车消费。2月16日，《求是》杂志发布的中央重要文章中指出，要稳定居民消费，鼓励汽车限购地区适当增加汽车号牌配额。2月20日，商务部在线上新闻发布会中，也提出鼓励汽车限购地区适当增加汽车号牌配额，带动汽车消费。3月13日，国家发改委、商务部等23个部门向各省市联合发布《关于促进消费扩容提质加快形成强大国内市场的实施意见》，提出要促进汽车限购向引导使用政策转变，鼓励汽车限购地区适当增加汽车号牌限额。而在刺激车市的措施中，放开部分城市的汽车限购在不少业内人士看来是最为直接，同时也是风险相对较低的做法。乘联会秘书长崔东树就指出，如果限购城市被限制的50万

### 簡化版 model training & evaluation

In [ ]:
model.train(5000)

Number of training data: 5000


  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/2500 [00:00<?, ?it/s]

In [ ]:
model.evaluate(1000)

Generating outputs:   0%|          | 0/125 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1000 [00:00<?, ?it/s]

Number of test data: 1000


{'rouge-1': {'r': 0.3419968233881038,
  'p': 0.418517013571272,
  'f': 0.3764073241520695},
 'rouge-2': {'r': 0.23560655416648008,
  'p': 0.2843251044770485,
  'f': 0.25768332054882703},
 'rouge-l': {'r': 0.2983934919642755,
  'p': 0.3667253292231124,
  'f': 0.32904933101510636}}

In [ ]:
src = "3 月 25 日 ， 杭 州 市 小 客 车 总 量 调 控 管 理 办 公 室 发 布 公 告 称 ， 根 据 省 、 市 党 委 政 府 关 于 加 快 提 振 汽 车 消 费 ， 促 进 经 济 发 展 相 关 工 作 的 要 求 ， 经 研 究 决 定 ， 2020 年 一 次 性 增 加 2 万 个 小 客 车 指 标 ， 均 为 摇 号 指 标 ， 将 在 今 年 上 半 年 配 置 完 毕 。 杭 州 市 小 客 车 总 量 调 控 管 理 办 公 室 发 布 的 《 关 于 2020 年 一 次 性 增 加 小 客 车 指 标 的 配 置 公 告 》 具 体 规 定 这 2 万 个 小 客 车 指 标 全 部 是 摇 号 指 标 。 其 中 ， 个 人 阶 梯 摇 号 15000 个 ， 将 于 2020 年 5 月 29 日 组 织 一 次 个 人 阶 梯 摇 号 ， 完 成 15000 个 指 标 的 配 置 ； 新 增 的 5000 个 县 （ 市 ） 指 标 将 在 2020 年 5 月 18 、 19 日 举 行 的 第 十 次 小 客 车 县 （ 市 ） 指 标 摇 号 活 动 中 与 原 定 指 标 配 额 合 并 投 放 。 分 配 至 各 县 （ 市 ） 的 指 标 比 例 具 体 为 建 德 市 1833 个 ， 淳 安 县 1667 个 ， 桐 庐 县 1500 个 。 据 悉 ， 杭 州 市 的 小 客 车 指 标 有 摇 号 和 竞 拍 两 种 配 置 方 式 ， 去 年 ， 全 年 杭 州 市 个 人 摇 号 指 标 达 58005 个 ， 此 次 新 增 的 2 万 个 指 标 占 比 大 约 为 34 . 5 % 。 而 杭 州 去 年 个 人 竞 价 的 指 标 数 量 为 14300 个 。 近 日 ， 浙 江 省 人 民 政 府 办 公 厅 发 布 《 关 于 提 振 消 费 促 进 经 济 稳 定 增 长 的 实 施 意 见 》 ， 其 中 一 条 是 要 释 放 城 乡 汽 车 消 费 潜 力 。 上 述 意 见 指 出 了 几 个 方 向 ， 第 一 条 便 是 鼓 励 杭 州 有 序 放 宽 汽 车 限 购 措 施 ， 制 定 汽 车 以 旧 换 新 和 下 乡 惠 农 政 策 ， 深 挖 农 村 汽 车 消 费 潜 力 。 值 得 注 意 的 是 ， 在 春 节 销 售 淡 季 叠 加 疫 情 影 响 ， 中 国 车 市 刚 刚 经 历 了 艰 难 的 开 局 。 据 乘 联 会 数 据 显 示 ， 2 月 我 国 狭 义 乘 用 车 市 场 产 量 和 批 发 销 量 分 别 为 21 . 5 万 辆 、 25 . 2 万 辆 ， 同 比 骤 减 80 . 6 % 、 78 . 5 % ， 销 量 规 模 相 当 于 2006 年 2 月 水 平 。 1 - 2 月 累 计 批 发 销 量 196 . 9 万 辆 ， 同 比 下 降 41 % ， 乘 联 会 也 再 次 调 低 了 全 年 的 销 量 预 测 增 长 率 到 - 8 % 。 在 此 背 景 之 下 ， 自 疫 情 爆 发 以 来 ， 国 家 层 面 已 多 次 发 文 鼓 励 汽 车 消 费 。 2 月 16 日 ， 《 求 是 》 杂 志 发 布 的 中 央 重 要 文 章 中 指 出 ， 要 稳 定 居 民 消 费 ， 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 配 额 。 2 月 20 日 ， 商 务 部 在 线 上 新 闻 发 布 会 中 ， 也 提 出 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 配 额 ， 带 动 汽 车 消 费 。 3 月 13 日 ， 国 家 发 改 委 、 商 务 部 等 23 个 部 门 向 各 省 市 联 合 发 布 《 关 于 促 进 消 费 扩 容 提 质   加 快 形 成 强 大 国 内 市 场 的 实 施 意 见 》 ， 提 出 要 促 进 汽 车 限 购 向 引 导 使 用 政 策 转 变 ， 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 限 额 。 而 在 刺 激 车 市 的 措 施 中 ， 放 开 部 分 城 市 的 汽 车 限 购 在 不 少 业 内 人 士 看 来 是 最 为 直 接 ， 同 时 也 是 风 险 相 对 较 低 的 做 法 。 乘 联 会 秘 书 长 崔 东 树 就 指 出 ， 如 果 限 购 城 市 被 限 制 的 50 万 - 60 万 个 牌 照 配 额 有 序 放 开 ， 可 能 会 带 来 20 万 - 30 万 辆 的 销 量 ， 对 提 振 车 市 、 促 进 消 费 信 心 恢 复 具 有 积 极 的 作 用 。 目 前 ， 全 国 实 行 限 购 的 省 市 有 北 京 、 上 海 、 广 州 、 石 家 庄 、 天 津 、 杭 州 、 深 圳 、 海 南 ， 这 些 城 市 居 民 购 车 要 先 通 过 摇 号 或 拍 牌 的 方 式 获 取 指 标 。 此 前 ， 有 消 息 传 出 ， 北 京 将 率 先 “ 松 绑 ” 汽 车 限 购 ， 包 括 释 放 10 万 辆 新 能 源 汽 车 指 标 在 内 的 多 项 措 施 。 但 在 3 月 25 日 ， 北 京 市 商 务 局 在 官 方 微 博 上 发 表 致 歉 声 明 称 ， 以 上 措 施 均 未 经 过 研 究 论 证 。 ( 本 文 来 自 澎 湃 新 闻 ， 更 多 原 创 资 讯 请 下 载 “ 澎 湃 新 闻 ” APP )".replace(" ", "")
prd = model.perform(src)
print("src: ", src)
print("prd:", prd)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

src:  3月25日，杭州市小客车总量调控管理办公室发布公告称，根据省、市党委政府关于加快提振汽车消费，促进经济发展相关工作的要求，经研究决定，2020年一次性增加2万个小客车指标，均为摇号指标，将在今年上半年配置完毕。杭州市小客车总量调控管理办公室发布的《关于2020年一次性增加小客车指标的配置公告》具体规定这2万个小客车指标全部是摇号指标。其中，个人阶梯摇号15000个，将于2020年5月29日组织一次个人阶梯摇号，完成15000个指标的配置；新增的5000个县（市）指标将在2020年5月18、19日举行的第十次小客车县（市）指标摇号活动中与原定指标配额合并投放。分配至各县（市）的指标比例具体为建德市1833个，淳安县1667个，桐庐县1500个。据悉，杭州市的小客车指标有摇号和竞拍两种配置方式，去年，全年杭州市个人摇号指标达58005个，此次新增的2万个指标占比大约为34.5%。而杭州去年个人竞价的指标数量为14300个。近日，浙江省人民政府办公厅发布《关于提振消费促进经济稳定增长的实施意见》，其中一条是要释放城乡汽车消费潜力。上述意见指出了几个方向，第一条便是鼓励杭州有序放宽汽车限购措施，制定汽车以旧换新和下乡惠农政策，深挖农村汽车消费潜力。值得注意的是，在春节销售淡季叠加疫情影响，中国车市刚刚经历了艰难的开局。据乘联会数据显示，2月我国狭义乘用车市场产量和批发销量分别为21.5万辆、25.2万辆，同比骤减80.6%、78.5%，销量规模相当于2006年2月水平。1-2月累计批发销量196.9万辆，同比下降41%，乘联会也再次调低了全年的销量预测增长率到-8%。在此背景之下，自疫情爆发以来，国家层面已多次发文鼓励汽车消费。2月16日，《求是》杂志发布的中央重要文章中指出，要稳定居民消费，鼓励汽车限购地区适当增加汽车号牌配额。2月20日，商务部在线上新闻发布会中，也提出鼓励汽车限购地区适当增加汽车号牌配额，带动汽车消费。3月13日，国家发改委、商务部等23个部门向各省市联合发布《关于促进消费扩容提质加快形成强大国内市场的实施意见》，提出要促进汽车限购向引导使用政策转变，鼓励汽车限购地区适当增加汽车号牌限额。而在刺激车市的措施中，放开部分城市的汽车限购在不少业内人士看来是最为直接，同时也是风险相对较低的做法。乘联会秘书长崔东树就指出，如果限购城市被限制的50万

In [ ]:
!mv /content/outputs/checkpoint-2500-epoch-1/ /content/drive/MyDrive/AI_School/model_summ/

## Homework

使用前面訓練好的簡化版模型進行預測，並繳交預測結果。(只要前1000筆)

＊＊＊請將預測結果下載，然後上傳到個人資料夾＊＊＊

In [ ]:
model.load_default_model("/content/drive/MyDrive/AI_School/model_summ/")

In [ ]:
def hand_in():
    data_dir = "./CLTS"
    data_type = "valid"
    max_seq_len = 500
    eval_data = []
    predictions = []
    with open(os.path.join(data_dir, "%s.src" % data_type)) as fsrc, open("task1.txt", "w", encoding="utf-8") as fout:
        # [TODO] 讀取 valid.src 檔，轉成轉成 model input 的形式進行進行predict，
        # (只要 predict 前1000筆)
        # 預測結果輸出為task1.txt，請將檔名改為task1_XXX.txt (XXX為學員編號)進行繳交。
        ##################################################
        for src in fsrc:
            src = src.replace(" ", "")[:max_seq_len]
            eval_data.append("summarize: %s" % src)
        predictions = model.model.predict(eval_data[:1000])
        ##################################################
        if model.lang == 'zh':
            predictions = [" ".join(list(p)) for p in predictions] 
        print("Number of valid data: %d" % len(predictions)) 
        for line in predictions:
            print(line, file=fout)


hand_in()

Generating outputs:   0%|          | 0/125 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1000 [00:00<?, ?it/s]

Number of valid data: 1000


## Exercise
觀察模型輸入輸出和分詞的影響。

In [ ]:
model.load_default_model("./CLTS_model/")

In [ ]:
from transformers import BertTokenizer, T5Tokenizer
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
# [TODO] 除了使用 mt5_tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")
# 還可以用前面 model = T5Summarizer() 的 model 直接使用 tokenizer
# mt5_tokenizer = model ...
##################################################
mt5_tokenizer = model.model.tokenizer
##################################################

In [ ]:
src = "3 月 25 日 ， 杭 州 市 小 客 车 总 量 调 控 管 理 办 公 室 发 布 公 告 称 ， 根 据 省 、 市 党 委 政 府 关 于 加 快 提 振 汽 车 消 费 ， 促 进 经 济 发 展 相 关 工 作 的 要 求 ， 经 研 究 决 定 ， 2020 年 一 次 性 增 加 2 万 个 小 客 车 指 标 ， 均 为 摇 号 指 标 ， 将 在 今 年 上 半 年 配 置 完 毕 。 杭 州 市 小 客 车 总 量 调 控 管 理 办 公 室 发 布 的 《 关 于 2020 年 一 次 性 增 加 小 客 车 指 标 的 配 置 公 告 》 具 体 规 定 这 2 万 个 小 客 车 指 标 全 部 是 摇 号 指 标 。 其 中 ， 个 人 阶 梯 摇 号 15000 个 ， 将 于 2020 年 5 月 29 日 组 织 一 次 个 人 阶 梯 摇 号 ， 完 成 15000 个 指 标 的 配 置 ； 新 增 的 5000 个 县 （ 市 ） 指 标 将 在 2020 年 5 月 18 、 19 日 举 行 的 第 十 次 小 客 车 县 （ 市 ） 指 标 摇 号 活 动 中 与 原 定 指 标 配 额 合 并 投 放 。 分 配 至 各 县 （ 市 ） 的 指 标 比 例 具 体 为 建 德 市 1833 个 ， 淳 安 县 1667 个 ， 桐 庐 县 1500 个 。 据 悉 ， 杭 州 市 的 小 客 车 指 标 有 摇 号 和 竞 拍 两 种 配 置 方 式 ， 去 年 ， 全 年 杭 州 市 个 人 摇 号 指 标 达 58005 个 ， 此 次 新 增 的 2 万 个 指 标 占 比 大 约 为 34 . 5 % 。 而 杭 州 去 年 个 人 竞 价 的 指 标 数 量 为 14300 个 。 近 日 ， 浙 江 省 人 民 政 府 办 公 厅 发 布 《 关 于 提 振 消 费 促 进 经 济 稳 定 增 长 的 实 施 意 见 》 ， 其 中 一 条 是 要 释 放 城 乡 汽 车 消 费 潜 力 。 上 述 意 见 指 出 了 几 个 方 向 ， 第 一 条 便 是 鼓 励 杭 州 有 序 放 宽 汽 车 限 购 措 施 ， 制 定 汽 车 以 旧 换 新 和 下 乡 惠 农 政 策 ， 深 挖 农 村 汽 车 消 费 潜 力 。 值 得 注 意 的 是 ， 在 春 节 销 售 淡 季 叠 加 疫 情 影 响 ， 中 国 车 市 刚 刚 经 历 了 艰 难 的 开 局 。 据 乘 联 会 数 据 显 示 ， 2 月 我 国 狭 义 乘 用 车 市 场 产 量 和 批 发 销 量 分 别 为 21 . 5 万 辆 、 25 . 2 万 辆 ， 同 比 骤 减 80 . 6 % 、 78 . 5 % ， 销 量 规 模 相 当 于 2006 年 2 月 水 平 。 1 - 2 月 累 计 批 发 销 量 196 . 9 万 辆 ， 同 比 下 降 41 % ， 乘 联 会 也 再 次 调 低 了 全 年 的 销 量 预 测 增 长 率 到 - 8 % 。 在 此 背 景 之 下 ， 自 疫 情 爆 发 以 来 ， 国 家 层 面 已 多 次 发 文 鼓 励 汽 车 消 费 。 2 月 16 日 ， 《 求 是 》 杂 志 发 布 的 中 央 重 要 文 章 中 指 出 ， 要 稳 定 居 民 消 费 ， 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 配 额 。 2 月 20 日 ， 商 务 部 在 线 上 新 闻 发 布 会 中 ， 也 提 出 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 配 额 ， 带 动 汽 车 消 费 。 3 月 13 日 ， 国 家 发 改 委 、 商 务 部 等 23 个 部 门 向 各 省 市 联 合 发 布 《 关 于 促 进 消 费 扩 容 提 质   加 快 形 成 强 大 国 内 市 场 的 实 施 意 见 》 ， 提 出 要 促 进 汽 车 限 购 向 引 导 使 用 政 策 转 变 ， 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 限 额 。 而 在 刺 激 车 市 的 措 施 中 ， 放 开 部 分 城 市 的 汽 车 限 购 在 不 少 业 内 人 士 看 来 是 最 为 直 接 ， 同 时 也 是 风 险 相 对 较 低 的 做 法 。 乘 联 会 秘 书 长 崔 东 树 就 指 出 ， 如 果 限 购 城 市 被 限 制 的 50 万 - 60 万 个 牌 照 配 额 有 序 放 开 ， 可 能 会 带 来 20 万 - 30 万 辆 的 销 量 ， 对 提 振 车 市 、 促 进 消 费 信 心 恢 复 具 有 积 极 的 作 用 。 目 前 ， 全 国 实 行 限 购 的 省 市 有 北 京 、 上 海 、 广 州 、 石 家 庄 、 天 津 、 杭 州 、 深 圳 、 海 南 ， 这 些 城 市 居 民 购 车 要 先 通 过 摇 号 或 拍 牌 的 方 式 获 取 指 标 。 此 前 ， 有 消 息 传 出 ， 北 京 将 率 先 “ 松 绑 ” 汽 车 限 购 ， 包 括 释 放 10 万 辆 新 能 源 汽 车 指 标 在 内 的 多 项 措 施 。 但 在 3 月 25 日 ， 北 京 市 商 务 局 在 官 方 微 博 上 发 表 致 歉 声 明 称 ， 以 上 措 施 均 未 经 过 研 究 论 证 。 ( 本 文 来 自 澎 湃 新 闻 ， 更 多 原 创 资 讯 请 下 载 “ 澎 湃 新 闻 ” APP )".replace(" ", "")
prd = model.perform(src)
# [TODO] 使用tokenizer.tokenize()來對src和prd做句子的分詞處理，比較bert_tokenizer和mt5_tokenizer兩種方法的差異。
##################################################
print("[src]")
print("bert: ", " ".join(bert_tokenizer.tokenize(src)))
print("mt5:  ", " ".join(mt5_tokenizer.tokenize(src)))
print("[prd]")
print("bert: ", " ".join(bert_tokenizer.tokenize(prd)))
print("mt5:  ", " ".join(mt5_tokenizer.tokenize(prd)))
##################################################

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

[src]
bert:  3 月 25 日 ， 杭 州 市 小 客 车 总 量 调 控 管 理 办 公 室 发 布 公 告 称 ， 根 据 省 、 市 党 委 政 府 关 于 加 快 提 振 汽 车 消 费 ， 促 进 经 济 发 展 相 关 工 作 的 要 求 ， 经 研 究 决 定 ， 2020 年 一 次 性 增 加 2 万 个 小 客 车 指 标 ， 均 为 摇 号 指 标 ， 将 在 今 年 上 半 年 配 置 完 毕 。 杭 州 市 小 客 车 总 量 调 控 管 理 办 公 室 发 布 的 《 关 于 2020 年 一 次 性 增 加 小 客 车 指 标 的 配 置 公 告 》 具 体 规 定 这 2 万 个 小 客 车 指 标 全 部 是 摇 号 指 标 。 其 中 ， 个 人 阶 梯 摇 号 15000 个 ， 将 于 2020 年 5 月 29 日 组 织 一 次 个 人 阶 梯 摇 号 ， 完 成 15000 个 指 标 的 配 置 ； 新 增 的 5000 个 县 （ 市 ） 指 标 将 在 2020 年 5 月 18 、 19 日 举 行 的 第 十 次 小 客 车 县 （ 市 ） 指 标 摇 号 活 动 中 与 原 定 指 标 配 额 合 并 投 放 。 分 配 至 各 县 （ 市 ） 的 指 标 比 例 具 体 为 建 德 市 183 ##3 个 ， 淳 安 县 166 ##7 个 ， 桐 庐 县 1500 个 。 据 悉 ， 杭 州 市 的 小 客 车 指 标 有 摇 号 和 竞 拍 两 种 配 置 方 式 ， 去 年 ， 全 年 杭 州 市 个 人 摇 号 指 标 达 580 ##05 个 ， 此 次 新 增 的 2 万 个 指 标 占 比 大 约 为 34 . 5 % 。 而 杭 州 去 年 个 人 竞 价 的 指 标 数 量 为 143 ##00 个 。 近 日 ， 浙 江 省 人 民 政 府 办 公 厅 发 布 《 关 于 提 振 消 费 促 进 经 济 稳 定 增 长 的 实 施 意 见 》 ， 其 中 一 条 是 要 释 放 城 乡 汽 车 消 费 潜 力 。 上 述 意 见 指 出 了 几 个 方 向 ， 第 一 条 便 是 鼓 励 杭 州 有 序 放 宽 汽 车 限 购 措 施 ， 制 定 汽 车 以 旧 换 新 和 

## Exercise
觀察模型輸入中的prefix對輸出的影響。

In [ ]:
# [TODO] 分別使用 "translate Chinese to English:", "generate question:", "summarize:" 作為作為input prefix，
# 並使用使用上面的 src 作為輸入來測試，比較使用三種不同 prefix 的 predict 結果。
##################################################
result_1 = model.model.predict(["translate Chinese to English: %s" % src])[0]
result_2 = model.model.predict(["generate question: %s" % src])[0]
result_3 = model.model.predict(["summarize: %s" % src])[0]
print("translate Chinese to English => ", result_1)
print("generate question => ", result_2)
print("summarize => ", result_3)
##################################################

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

translate Chinese to English =>  近日,浙江省人民政府办公厅发布《关于提振消费促进经济稳定增长的实施意见》,其中一条是要释放城乡汽车消费潜力。
generate question =>  近日,浙江省人民政府办公厅发布《关于提振消费促进经济稳定增长的实施意见》,其中一条是要释放城乡汽车消费潜力。
summarize =>  杭州市小客车总量调控管理办公室发布公告称,根据省、市党委政府关于加快提振汽车消费,促进经济发展相关工作的要求,经研究决定,2020年一次性增加2万个小客车指标。


## Exercise
指定較少的訓練資料（例如僅用前一千筆 ```model.train(1000)``` ），觀察測試資料與效能的關係。

In [ ]:
# [TODO] 分別使用 1000, 2000, 3000 筆 training data 來訓練 model，
# 並使用使用 1000 筆 test data 來測試，比較三種不同訓練資料大小 model 的 rouge。
##################################################
model.train(1000)
result_1 = model.evaluate(1000)
model.train(2000)
result_2 = model.evaluate(1000)
model.train(3000)
result_3 = model.evaluate(1000)
print("train_size=1000 => ", result_1)
print("train_size=2000 => ", result_2)
print("train_size=3000 => ", result_3)
##################################################

Number of training data: 1000


  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/500 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1500 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/125 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1000 [00:00<?, ?it/s]

Number of test data: 1000
train_size=1000 =>  {'rouge-1': {'r': 0.2435189029670833, 'p': 0.3505731946418963, 'f': 0.2874005566222835}, 'rouge-2': {'r': 0.1433723746375254, 'p': 0.19675561670069977, 'f': 0.16587473367693853}, 'rouge-l': {'r': 0.20501545049746922, 'p': 0.2977528070796088, 'f': 0.24283126454519083}}
train_size=2000 =>  {'rouge-1': {'r': 0.2705216760904974, 'p': 0.4001331365824235, 'f': 0.32280298216667297}, 'rouge-2': {'r': 0.16992579920208667, 'p': 0.24577210788203166, 'f': 0.2009296710025762}, 'rouge-l': {'r': 0.2308832051078671, 'p': 0.3433245008223608, 'f': 0.2760947313081064}}
train_size=3000 =>  {'rouge-1': {'r': 0.3536977299586057, 'p': 0.36645548296668273, 'f': 0.35996360254980053}, 'rouge-2': {'r': 0.2365423781440623, 'p': 0.24061315404436287, 'f': 0.23856040150833585}, 'rouge-l': {'r': 0.3025042981844682, 'p': 0.3150717641565014, 'f': 0.3086601593093577}}


## Exercise
嘗試不同的 epoch 數，觀察 epoch 數與效能的關係。


In [ ]:
# [TODO] 使用 3000 筆 training data 和 3 個 epoch 來訓練 model，
# 並使用使用 1000 筆 test data 來測試，比較 epoch 分別為 1, 2, 3 時 model evaluate 的 rouge。
##################################################
model.train(3000)
epoch_paths = {"Epoch-1": "./outputs/checkpoint-1500-epoch-1/", 
               "Epoch-2": "./outputs/checkpoint-3000-epoch-2/",
               "Epoch-3": "./outputs/checkpoint-4500-epoch-3/",
              }
for model_epoch, model_path in epoch_paths.items():
    model.load_default_model(model_path)
    result = model.evaluate(1000)
    print("%s => %s" % (model_epoch, result))
##################################################

Number of training data: 3000


  0%|          | 0/3000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/1500 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/1500 [00:00<?, ?it/s]

...

Decoding outputs:   0%|          | 0/1000 [00:00<?, ?it/s]

Number of test data: 1000
Epoch-2 => {'rouge-1': {'r': 0.4245284960967893, 'p': 0.4111098426028613, 'f': 0.41771143119729487}, 'rouge-2': {'r': 0.3121100224187957, 'p': 0.2947559111529411, 'f': 0.30318483522898965}, 'rouge-l': {'r': 0.37422009620132307, 'p': 0.3607197232847113, 'f': 0.3673459131489729}}


Generating outputs:   0%|          | 0/125 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1000 [00:00<?, ?it/s]

Number of test data: 1000
Epoch-3 => {'rouge-1': {'r': 0.5016548993277868, 'p': 0.4130963744429198, 'f': 0.45308889110283285}, 'rouge-2': {'r': 0.3759117241943057, 'p': 0.30999553075196357, 'f': 0.33978633005311487}, 'rouge-l': {'r': 0.4410383594405045, 'p': 0.3641907257589306, 'f': 0.3989475372027412}}


## Exercise*
嘗試投影片中提到的 decoding 方法。
(Hint: 改變self.model_args的參數設置，可以參考Simple Transformers的Documentation)

In [ ]:
class T5NewSummarizer(T5Summarizer):
    def create_model(self):
        self.model_args = T5Args()
        self.model_args.max_seq_length = self.max_seq_len
        self.model_args.max_length = self.max_output_len
        self.model_args.train_batch_size = 2
        self.model_args.eval_batch_size = 8
        self.model_args.num_train_epochs = 1
        self.model_args.evaluate_during_training = False
        self.model_args.evaluate_during_training_steps = 300000
        self.model_args.use_multiprocessing = False
        self.model_args.fp16 = False
        self.model_args.save_steps = -1
        self.model_args.save_eval_checkpoints = False
        self.model_args.no_cache = True
        self.model_args.reprocess_input_data = True
        self.model_args.overwrite_output_dir = True
        ##################################################
        # self.model_args.do_sample = False
        # self.model_args.top_k = None
        # self.model_args.top_p = None
        # self.model_args.num_beams = 1
        self.model_args.num_return_sequences = 1
        ##################################################
        self.model_args.output_dir = "outputs/"
        self.model = T5Model("mt5", "./CLTS_model/", args=self.model_args, use_cuda=True)
        print(self.model_args)

model = T5NewSummarizer()


In [ ]:
model.create_model()

T5Args(adafactor_beta1=None, adafactor_clip_threshold=1.0, adafactor_decay_rate=-0.8, adafactor_eps=(1e-30, 0.001), adafactor_relative_step=False, adafactor_scale_parameter=False, adafactor_warmup_init=False, adam_epsilon=1e-08, best_model_dir='outputs/best_model', cache_dir='cache_dir/', config={}, cosine_schedule_num_cycles=0.5, custom_layer_parameters=[], custom_parameter_groups=[], dataloader_num_workers=0, do_lower_case=False, dynamic_quantize=False, early_stopping_consider_epochs=False, early_stopping_delta=0, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=8, evaluate_during_training=False, evaluate_during_training_silent=True, evaluate_during_training_steps=300000, evaluate_during_training_verbose=False, evaluate_each_epoch=True, fp16=False, gradient_accumulation_steps=1, learning_rate=0.001, local_rank=-1, logging_steps=50, loss_type=None, loss_args={}, manual_seed=None, max_grad_norm=1.0, max_se

In [ ]:
model.evaluate(1000)

Generating outputs:   0%|          | 0/125 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1000 [00:00<?, ?it/s]

Number of test data: 1000


{'rouge-1': {'r': 0.462611590930758,
  'p': 0.4490039358563163,
  'f': 0.4557062028830018},
 'rouge-2': {'r': 0.34696309709116047,
  'p': 0.33757198196663013,
  'f': 0.3422031213230353},
 'rouge-l': {'r': 0.41051342228473886,
  'p': 0.3985519577663,
  'f': 0.40444426909134423}}

In [ ]:
src = "3 月 25 日 ， 杭 州 市 小 客 车 总 量 调 控 管 理 办 公 室 发 布 公 告 称 ， 根 据 省 、 市 党 委 政 府 关 于 加 快 提 振 汽 车 消 费 ， 促 进 经 济 发 展 相 关 工 作 的 要 求 ， 经 研 究 决 定 ， 2020 年 一 次 性 增 加 2 万 个 小 客 车 指 标 ， 均 为 摇 号 指 标 ， 将 在 今 年 上 半 年 配 置 完 毕 。 杭 州 市 小 客 车 总 量 调 控 管 理 办 公 室 发 布 的 《 关 于 2020 年 一 次 性 增 加 小 客 车 指 标 的 配 置 公 告 》 具 体 规 定 这 2 万 个 小 客 车 指 标 全 部 是 摇 号 指 标 。 其 中 ， 个 人 阶 梯 摇 号 15000 个 ， 将 于 2020 年 5 月 29 日 组 织 一 次 个 人 阶 梯 摇 号 ， 完 成 15000 个 指 标 的 配 置 ； 新 增 的 5000 个 县 （ 市 ） 指 标 将 在 2020 年 5 月 18 、 19 日 举 行 的 第 十 次 小 客 车 县 （ 市 ） 指 标 摇 号 活 动 中 与 原 定 指 标 配 额 合 并 投 放 。 分 配 至 各 县 （ 市 ） 的 指 标 比 例 具 体 为 建 德 市 1833 个 ， 淳 安 县 1667 个 ， 桐 庐 县 1500 个 。 据 悉 ， 杭 州 市 的 小 客 车 指 标 有 摇 号 和 竞 拍 两 种 配 置 方 式 ， 去 年 ， 全 年 杭 州 市 个 人 摇 号 指 标 达 58005 个 ， 此 次 新 增 的 2 万 个 指 标 占 比 大 约 为 34 . 5 % 。 而 杭 州 去 年 个 人 竞 价 的 指 标 数 量 为 14300 个 。 近 日 ， 浙 江 省 人 民 政 府 办 公 厅 发 布 《 关 于 提 振 消 费 促 进 经 济 稳 定 增 长 的 实 施 意 见 》 ， 其 中 一 条 是 要 释 放 城 乡 汽 车 消 费 潜 力 。 上 述 意 见 指 出 了 几 个 方 向 ， 第 一 条 便 是 鼓 励 杭 州 有 序 放 宽 汽 车 限 购 措 施 ， 制 定 汽 车 以 旧 换 新 和 下 乡 惠 农 政 策 ， 深 挖 农 村 汽 车 消 费 潜 力 。 值 得 注 意 的 是 ， 在 春 节 销 售 淡 季 叠 加 疫 情 影 响 ， 中 国 车 市 刚 刚 经 历 了 艰 难 的 开 局 。 据 乘 联 会 数 据 显 示 ， 2 月 我 国 狭 义 乘 用 车 市 场 产 量 和 批 发 销 量 分 别 为 21 . 5 万 辆 、 25 . 2 万 辆 ， 同 比 骤 减 80 . 6 % 、 78 . 5 % ， 销 量 规 模 相 当 于 2006 年 2 月 水 平 。 1 - 2 月 累 计 批 发 销 量 196 . 9 万 辆 ， 同 比 下 降 41 % ， 乘 联 会 也 再 次 调 低 了 全 年 的 销 量 预 测 增 长 率 到 - 8 % 。 在 此 背 景 之 下 ， 自 疫 情 爆 发 以 来 ， 国 家 层 面 已 多 次 发 文 鼓 励 汽 车 消 费 。 2 月 16 日 ， 《 求 是 》 杂 志 发 布 的 中 央 重 要 文 章 中 指 出 ， 要 稳 定 居 民 消 费 ， 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 配 额 。 2 月 20 日 ， 商 务 部 在 线 上 新 闻 发 布 会 中 ， 也 提 出 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 配 额 ， 带 动 汽 车 消 费 。 3 月 13 日 ， 国 家 发 改 委 、 商 务 部 等 23 个 部 门 向 各 省 市 联 合 发 布 《 关 于 促 进 消 费 扩 容 提 质   加 快 形 成 强 大 国 内 市 场 的 实 施 意 见 》 ， 提 出 要 促 进 汽 车 限 购 向 引 导 使 用 政 策 转 变 ， 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 限 额 。 而 在 刺 激 车 市 的 措 施 中 ， 放 开 部 分 城 市 的 汽 车 限 购 在 不 少 业 内 人 士 看 来 是 最 为 直 接 ， 同 时 也 是 风 险 相 对 较 低 的 做 法 。 乘 联 会 秘 书 长 崔 东 树 就 指 出 ， 如 果 限 购 城 市 被 限 制 的 50 万 - 60 万 个 牌 照 配 额 有 序 放 开 ， 可 能 会 带 来 20 万 - 30 万 辆 的 销 量 ， 对 提 振 车 市 、 促 进 消 费 信 心 恢 复 具 有 积 极 的 作 用 。 目 前 ， 全 国 实 行 限 购 的 省 市 有 北 京 、 上 海 、 广 州 、 石 家 庄 、 天 津 、 杭 州 、 深 圳 、 海 南 ， 这 些 城 市 居 民 购 车 要 先 通 过 摇 号 或 拍 牌 的 方 式 获 取 指 标 。 此 前 ， 有 消 息 传 出 ， 北 京 将 率 先 “ 松 绑 ” 汽 车 限 购 ， 包 括 释 放 10 万 辆 新 能 源 汽 车 指 标 在 内 的 多 项 措 施 。 但 在 3 月 25 日 ， 北 京 市 商 务 局 在 官 方 微 博 上 发 表 致 歉 声 明 称 ， 以 上 措 施 均 未 经 过 研 究 论 证 。 ( 本 文 来 自 澎 湃 新 闻 ， 更 多 原 创 资 讯 请 下 载 “ 澎 湃 新 闻 ” APP )".replace(" ", "")
prds = model.perform(src)
print("src: ", src)
print("prd: ", prd)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

src:  3月25日，杭州市小客车总量调控管理办公室发布公告称，根据省、市党委政府关于加快提振汽车消费，促进经济发展相关工作的要求，经研究决定，2020年一次性增加2万个小客车指标，均为摇号指标，将在今年上半年配置完毕。杭州市小客车总量调控管理办公室发布的《关于2020年一次性增加小客车指标的配置公告》具体规定这2万个小客车指标全部是摇号指标。其中，个人阶梯摇号15000个，将于2020年5月29日组织一次个人阶梯摇号，完成15000个指标的配置；新增的5000个县（市）指标将在2020年5月18、19日举行的第十次小客车县（市）指标摇号活动中与原定指标配额合并投放。分配至各县（市）的指标比例具体为建德市1833个，淳安县1667个，桐庐县1500个。据悉，杭州市的小客车指标有摇号和竞拍两种配置方式，去年，全年杭州市个人摇号指标达58005个，此次新增的2万个指标占比大约为34.5%。而杭州去年个人竞价的指标数量为14300个。近日，浙江省人民政府办公厅发布《关于提振消费促进经济稳定增长的实施意见》，其中一条是要释放城乡汽车消费潜力。上述意见指出了几个方向，第一条便是鼓励杭州有序放宽汽车限购措施，制定汽车以旧换新和下乡惠农政策，深挖农村汽车消费潜力。值得注意的是，在春节销售淡季叠加疫情影响，中国车市刚刚经历了艰难的开局。据乘联会数据显示，2月我国狭义乘用车市场产量和批发销量分别为21.5万辆、25.2万辆，同比骤减80.6%、78.5%，销量规模相当于2006年2月水平。1-2月累计批发销量196.9万辆，同比下降41%，乘联会也再次调低了全年的销量预测增长率到-8%。在此背景之下，自疫情爆发以来，国家层面已多次发文鼓励汽车消费。2月16日，《求是》杂志发布的中央重要文章中指出，要稳定居民消费，鼓励汽车限购地区适当增加汽车号牌配额。2月20日，商务部在线上新闻发布会中，也提出鼓励汽车限购地区适当增加汽车号牌配额，带动汽车消费。3月13日，国家发改委、商务部等23个部门向各省市联合发布《关于促进消费扩容提质加快形成强大国内市场的实施意见》，提出要促进汽车限购向引导使用政策转变，鼓励汽车限购地区适当增加汽车号牌限额。而在刺激车市的措施中，放开部分城市的汽车限购在不少业内人士看来是最为直接，同时也是风险相对较低的做法。乘联会秘书长崔东树就指出，如果限购城市被限制的50万

#### Beam Search

* self.model_args.num_beams = 3

In [ ]:
model.create_model()

T5Args(adafactor_beta1=None, adafactor_clip_threshold=1.0, adafactor_decay_rate=-0.8, adafactor_eps=(1e-30, 0.001), adafactor_relative_step=False, adafactor_scale_parameter=False, adafactor_warmup_init=False, adam_epsilon=1e-08, best_model_dir='outputs/best_model', cache_dir='cache_dir/', config={}, cosine_schedule_num_cycles=0.5, custom_layer_parameters=[], custom_parameter_groups=[], dataloader_num_workers=0, do_lower_case=False, dynamic_quantize=False, early_stopping_consider_epochs=False, early_stopping_delta=0, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=8, evaluate_during_training=False, evaluate_during_training_silent=True, evaluate_during_training_steps=300000, evaluate_during_training_verbose=False, evaluate_each_epoch=True, fp16=False, gradient_accumulation_steps=1, learning_rate=0.001, local_rank=-1, logging_steps=50, loss_type=None, loss_args={}, manual_seed=None, max_grad_norm=1.0, max_se

In [ ]:
model.evaluate(1000)

Generating outputs:   0%|          | 0/125 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1000 [00:00<?, ?it/s]

Number of test data: 1000


{'rouge-1': {'r': 0.46369179212945905,
  'p': 0.46663212320430975,
  'f': 0.4651573111417939},
 'rouge-2': {'r': 0.3503011163427603,
  'p': 0.3514535949244691,
  'f': 0.3508764092866505},
 'rouge-l': {'r': 0.41088081987337727,
  'p': 0.41273920081774856,
  'f': 0.41180791375997267}}

In [ ]:
src = "3 月 25 日 ， 杭 州 市 小 客 车 总 量 调 控 管 理 办 公 室 发 布 公 告 称 ， 根 据 省 、 市 党 委 政 府 关 于 加 快 提 振 汽 车 消 费 ， 促 进 经 济 发 展 相 关 工 作 的 要 求 ， 经 研 究 决 定 ， 2020 年 一 次 性 增 加 2 万 个 小 客 车 指 标 ， 均 为 摇 号 指 标 ， 将 在 今 年 上 半 年 配 置 完 毕 。 杭 州 市 小 客 车 总 量 调 控 管 理 办 公 室 发 布 的 《 关 于 2020 年 一 次 性 增 加 小 客 车 指 标 的 配 置 公 告 》 具 体 规 定 这 2 万 个 小 客 车 指 标 全 部 是 摇 号 指 标 。 其 中 ， 个 人 阶 梯 摇 号 15000 个 ， 将 于 2020 年 5 月 29 日 组 织 一 次 个 人 阶 梯 摇 号 ， 完 成 15000 个 指 标 的 配 置 ； 新 增 的 5000 个 县 （ 市 ） 指 标 将 在 2020 年 5 月 18 、 19 日 举 行 的 第 十 次 小 客 车 县 （ 市 ） 指 标 摇 号 活 动 中 与 原 定 指 标 配 额 合 并 投 放 。 分 配 至 各 县 （ 市 ） 的 指 标 比 例 具 体 为 建 德 市 1833 个 ， 淳 安 县 1667 个 ， 桐 庐 县 1500 个 。 据 悉 ， 杭 州 市 的 小 客 车 指 标 有 摇 号 和 竞 拍 两 种 配 置 方 式 ， 去 年 ， 全 年 杭 州 市 个 人 摇 号 指 标 达 58005 个 ， 此 次 新 增 的 2 万 个 指 标 占 比 大 约 为 34 . 5 % 。 而 杭 州 去 年 个 人 竞 价 的 指 标 数 量 为 14300 个 。 近 日 ， 浙 江 省 人 民 政 府 办 公 厅 发 布 《 关 于 提 振 消 费 促 进 经 济 稳 定 增 长 的 实 施 意 见 》 ， 其 中 一 条 是 要 释 放 城 乡 汽 车 消 费 潜 力 。 上 述 意 见 指 出 了 几 个 方 向 ， 第 一 条 便 是 鼓 励 杭 州 有 序 放 宽 汽 车 限 购 措 施 ， 制 定 汽 车 以 旧 换 新 和 下 乡 惠 农 政 策 ， 深 挖 农 村 汽 车 消 费 潜 力 。 值 得 注 意 的 是 ， 在 春 节 销 售 淡 季 叠 加 疫 情 影 响 ， 中 国 车 市 刚 刚 经 历 了 艰 难 的 开 局 。 据 乘 联 会 数 据 显 示 ， 2 月 我 国 狭 义 乘 用 车 市 场 产 量 和 批 发 销 量 分 别 为 21 . 5 万 辆 、 25 . 2 万 辆 ， 同 比 骤 减 80 . 6 % 、 78 . 5 % ， 销 量 规 模 相 当 于 2006 年 2 月 水 平 。 1 - 2 月 累 计 批 发 销 量 196 . 9 万 辆 ， 同 比 下 降 41 % ， 乘 联 会 也 再 次 调 低 了 全 年 的 销 量 预 测 增 长 率 到 - 8 % 。 在 此 背 景 之 下 ， 自 疫 情 爆 发 以 来 ， 国 家 层 面 已 多 次 发 文 鼓 励 汽 车 消 费 。 2 月 16 日 ， 《 求 是 》 杂 志 发 布 的 中 央 重 要 文 章 中 指 出 ， 要 稳 定 居 民 消 费 ， 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 配 额 。 2 月 20 日 ， 商 务 部 在 线 上 新 闻 发 布 会 中 ， 也 提 出 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 配 额 ， 带 动 汽 车 消 费 。 3 月 13 日 ， 国 家 发 改 委 、 商 务 部 等 23 个 部 门 向 各 省 市 联 合 发 布 《 关 于 促 进 消 费 扩 容 提 质   加 快 形 成 强 大 国 内 市 场 的 实 施 意 见 》 ， 提 出 要 促 进 汽 车 限 购 向 引 导 使 用 政 策 转 变 ， 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 限 额 。 而 在 刺 激 车 市 的 措 施 中 ， 放 开 部 分 城 市 的 汽 车 限 购 在 不 少 业 内 人 士 看 来 是 最 为 直 接 ， 同 时 也 是 风 险 相 对 较 低 的 做 法 。 乘 联 会 秘 书 长 崔 东 树 就 指 出 ， 如 果 限 购 城 市 被 限 制 的 50 万 - 60 万 个 牌 照 配 额 有 序 放 开 ， 可 能 会 带 来 20 万 - 30 万 辆 的 销 量 ， 对 提 振 车 市 、 促 进 消 费 信 心 恢 复 具 有 积 极 的 作 用 。 目 前 ， 全 国 实 行 限 购 的 省 市 有 北 京 、 上 海 、 广 州 、 石 家 庄 、 天 津 、 杭 州 、 深 圳 、 海 南 ， 这 些 城 市 居 民 购 车 要 先 通 过 摇 号 或 拍 牌 的 方 式 获 取 指 标 。 此 前 ， 有 消 息 传 出 ， 北 京 将 率 先 “ 松 绑 ” 汽 车 限 购 ， 包 括 释 放 10 万 辆 新 能 源 汽 车 指 标 在 内 的 多 项 措 施 。 但 在 3 月 25 日 ， 北 京 市 商 务 局 在 官 方 微 博 上 发 表 致 歉 声 明 称 ， 以 上 措 施 均 未 经 过 研 究 论 证 。 ( 本 文 来 自 澎 湃 新 闻 ， 更 多 原 创 资 讯 请 下 载 “ 澎 湃 新 闻 ” APP )".replace(" ", "")
prds = model.perform(src)
print("src: ", src)
print("prd: ", prd)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

src:  3月25日，杭州市小客车总量调控管理办公室发布公告称，根据省、市党委政府关于加快提振汽车消费，促进经济发展相关工作的要求，经研究决定，2020年一次性增加2万个小客车指标，均为摇号指标，将在今年上半年配置完毕。杭州市小客车总量调控管理办公室发布的《关于2020年一次性增加小客车指标的配置公告》具体规定这2万个小客车指标全部是摇号指标。其中，个人阶梯摇号15000个，将于2020年5月29日组织一次个人阶梯摇号，完成15000个指标的配置；新增的5000个县（市）指标将在2020年5月18、19日举行的第十次小客车县（市）指标摇号活动中与原定指标配额合并投放。分配至各县（市）的指标比例具体为建德市1833个，淳安县1667个，桐庐县1500个。据悉，杭州市的小客车指标有摇号和竞拍两种配置方式，去年，全年杭州市个人摇号指标达58005个，此次新增的2万个指标占比大约为34.5%。而杭州去年个人竞价的指标数量为14300个。近日，浙江省人民政府办公厅发布《关于提振消费促进经济稳定增长的实施意见》，其中一条是要释放城乡汽车消费潜力。上述意见指出了几个方向，第一条便是鼓励杭州有序放宽汽车限购措施，制定汽车以旧换新和下乡惠农政策，深挖农村汽车消费潜力。值得注意的是，在春节销售淡季叠加疫情影响，中国车市刚刚经历了艰难的开局。据乘联会数据显示，2月我国狭义乘用车市场产量和批发销量分别为21.5万辆、25.2万辆，同比骤减80.6%、78.5%，销量规模相当于2006年2月水平。1-2月累计批发销量196.9万辆，同比下降41%，乘联会也再次调低了全年的销量预测增长率到-8%。在此背景之下，自疫情爆发以来，国家层面已多次发文鼓励汽车消费。2月16日，《求是》杂志发布的中央重要文章中指出，要稳定居民消费，鼓励汽车限购地区适当增加汽车号牌配额。2月20日，商务部在线上新闻发布会中，也提出鼓励汽车限购地区适当增加汽车号牌配额，带动汽车消费。3月13日，国家发改委、商务部等23个部门向各省市联合发布《关于促进消费扩容提质加快形成强大国内市场的实施意见》，提出要促进汽车限购向引导使用政策转变，鼓励汽车限购地区适当增加汽车号牌限额。而在刺激车市的措施中，放开部分城市的汽车限购在不少业内人士看来是最为直接，同时也是风险相对较低的做法。乘联会秘书长崔东树就指出，如果限购城市被限制的50万

#### Beam Search + Sampling
* self.model_args.do_sample = True
* self.model_args.top_k = 5
* self.model_args.top_p = 0.8
* self.model_args.num_beams = 3

In [ ]:
model.create_model()

T5Args(adafactor_beta1=None, adafactor_clip_threshold=1.0, adafactor_decay_rate=-0.8, adafactor_eps=(1e-30, 0.001), adafactor_relative_step=False, adafactor_scale_parameter=False, adafactor_warmup_init=False, adam_epsilon=1e-08, best_model_dir='outputs/best_model', cache_dir='cache_dir/', config={}, cosine_schedule_num_cycles=0.5, custom_layer_parameters=[], custom_parameter_groups=[], dataloader_num_workers=0, do_lower_case=False, dynamic_quantize=False, early_stopping_consider_epochs=False, early_stopping_delta=0, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=8, evaluate_during_training=False, evaluate_during_training_silent=True, evaluate_during_training_steps=300000, evaluate_during_training_verbose=False, evaluate_each_epoch=True, fp16=False, gradient_accumulation_steps=1, learning_rate=0.001, local_rank=-1, logging_steps=50, loss_type=None, loss_args={}, manual_seed=None, max_grad_norm=1.0, max_se

In [ ]:
model.evaluate(1000)

Generating outputs:   0%|          | 0/125 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/1000 [00:00<?, ?it/s]

Number of test data: 1000


{'rouge-1': {'r': 0.4710478200707321,
  'p': 0.4577685006598102,
  'f': 0.4643132329183252},
 'rouge-2': {'r': 0.3546289930255167,
  'p': 0.3444756459286575,
  'f': 0.3494785891286349},
 'rouge-l': {'r': 0.4175292958486349,
  'p': 0.4057052994787403,
  'f': 0.41153238451077295}}

In [ ]:
src = "3 月 25 日 ， 杭 州 市 小 客 车 总 量 调 控 管 理 办 公 室 发 布 公 告 称 ， 根 据 省 、 市 党 委 政 府 关 于 加 快 提 振 汽 车 消 费 ， 促 进 经 济 发 展 相 关 工 作 的 要 求 ， 经 研 究 决 定 ， 2020 年 一 次 性 增 加 2 万 个 小 客 车 指 标 ， 均 为 摇 号 指 标 ， 将 在 今 年 上 半 年 配 置 完 毕 。 杭 州 市 小 客 车 总 量 调 控 管 理 办 公 室 发 布 的 《 关 于 2020 年 一 次 性 增 加 小 客 车 指 标 的 配 置 公 告 》 具 体 规 定 这 2 万 个 小 客 车 指 标 全 部 是 摇 号 指 标 。 其 中 ， 个 人 阶 梯 摇 号 15000 个 ， 将 于 2020 年 5 月 29 日 组 织 一 次 个 人 阶 梯 摇 号 ， 完 成 15000 个 指 标 的 配 置 ； 新 增 的 5000 个 县 （ 市 ） 指 标 将 在 2020 年 5 月 18 、 19 日 举 行 的 第 十 次 小 客 车 县 （ 市 ） 指 标 摇 号 活 动 中 与 原 定 指 标 配 额 合 并 投 放 。 分 配 至 各 县 （ 市 ） 的 指 标 比 例 具 体 为 建 德 市 1833 个 ， 淳 安 县 1667 个 ， 桐 庐 县 1500 个 。 据 悉 ， 杭 州 市 的 小 客 车 指 标 有 摇 号 和 竞 拍 两 种 配 置 方 式 ， 去 年 ， 全 年 杭 州 市 个 人 摇 号 指 标 达 58005 个 ， 此 次 新 增 的 2 万 个 指 标 占 比 大 约 为 34 . 5 % 。 而 杭 州 去 年 个 人 竞 价 的 指 标 数 量 为 14300 个 。 近 日 ， 浙 江 省 人 民 政 府 办 公 厅 发 布 《 关 于 提 振 消 费 促 进 经 济 稳 定 增 长 的 实 施 意 见 》 ， 其 中 一 条 是 要 释 放 城 乡 汽 车 消 费 潜 力 。 上 述 意 见 指 出 了 几 个 方 向 ， 第 一 条 便 是 鼓 励 杭 州 有 序 放 宽 汽 车 限 购 措 施 ， 制 定 汽 车 以 旧 换 新 和 下 乡 惠 农 政 策 ， 深 挖 农 村 汽 车 消 费 潜 力 。 值 得 注 意 的 是 ， 在 春 节 销 售 淡 季 叠 加 疫 情 影 响 ， 中 国 车 市 刚 刚 经 历 了 艰 难 的 开 局 。 据 乘 联 会 数 据 显 示 ， 2 月 我 国 狭 义 乘 用 车 市 场 产 量 和 批 发 销 量 分 别 为 21 . 5 万 辆 、 25 . 2 万 辆 ， 同 比 骤 减 80 . 6 % 、 78 . 5 % ， 销 量 规 模 相 当 于 2006 年 2 月 水 平 。 1 - 2 月 累 计 批 发 销 量 196 . 9 万 辆 ， 同 比 下 降 41 % ， 乘 联 会 也 再 次 调 低 了 全 年 的 销 量 预 测 增 长 率 到 - 8 % 。 在 此 背 景 之 下 ， 自 疫 情 爆 发 以 来 ， 国 家 层 面 已 多 次 发 文 鼓 励 汽 车 消 费 。 2 月 16 日 ， 《 求 是 》 杂 志 发 布 的 中 央 重 要 文 章 中 指 出 ， 要 稳 定 居 民 消 费 ， 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 配 额 。 2 月 20 日 ， 商 务 部 在 线 上 新 闻 发 布 会 中 ， 也 提 出 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 配 额 ， 带 动 汽 车 消 费 。 3 月 13 日 ， 国 家 发 改 委 、 商 务 部 等 23 个 部 门 向 各 省 市 联 合 发 布 《 关 于 促 进 消 费 扩 容 提 质   加 快 形 成 强 大 国 内 市 场 的 实 施 意 见 》 ， 提 出 要 促 进 汽 车 限 购 向 引 导 使 用 政 策 转 变 ， 鼓 励 汽 车 限 购 地 区 适 当 增 加 汽 车 号 牌 限 额 。 而 在 刺 激 车 市 的 措 施 中 ， 放 开 部 分 城 市 的 汽 车 限 购 在 不 少 业 内 人 士 看 来 是 最 为 直 接 ， 同 时 也 是 风 险 相 对 较 低 的 做 法 。 乘 联 会 秘 书 长 崔 东 树 就 指 出 ， 如 果 限 购 城 市 被 限 制 的 50 万 - 60 万 个 牌 照 配 额 有 序 放 开 ， 可 能 会 带 来 20 万 - 30 万 辆 的 销 量 ， 对 提 振 车 市 、 促 进 消 费 信 心 恢 复 具 有 积 极 的 作 用 。 目 前 ， 全 国 实 行 限 购 的 省 市 有 北 京 、 上 海 、 广 州 、 石 家 庄 、 天 津 、 杭 州 、 深 圳 、 海 南 ， 这 些 城 市 居 民 购 车 要 先 通 过 摇 号 或 拍 牌 的 方 式 获 取 指 标 。 此 前 ， 有 消 息 传 出 ， 北 京 将 率 先 “ 松 绑 ” 汽 车 限 购 ， 包 括 释 放 10 万 辆 新 能 源 汽 车 指 标 在 内 的 多 项 措 施 。 但 在 3 月 25 日 ， 北 京 市 商 务 局 在 官 方 微 博 上 发 表 致 歉 声 明 称 ， 以 上 措 施 均 未 经 过 研 究 论 证 。 ( 本 文 来 自 澎 湃 新 闻 ， 更 多 原 创 资 讯 请 下 载 “ 澎 湃 新 闻 ” APP )".replace(" ", "")
prds = model.perform(src)
print("src: ", src)
print("prd: ", prd)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

src:  3月25日，杭州市小客车总量调控管理办公室发布公告称，根据省、市党委政府关于加快提振汽车消费，促进经济发展相关工作的要求，经研究决定，2020年一次性增加2万个小客车指标，均为摇号指标，将在今年上半年配置完毕。杭州市小客车总量调控管理办公室发布的《关于2020年一次性增加小客车指标的配置公告》具体规定这2万个小客车指标全部是摇号指标。其中，个人阶梯摇号15000个，将于2020年5月29日组织一次个人阶梯摇号，完成15000个指标的配置；新增的5000个县（市）指标将在2020年5月18、19日举行的第十次小客车县（市）指标摇号活动中与原定指标配额合并投放。分配至各县（市）的指标比例具体为建德市1833个，淳安县1667个，桐庐县1500个。据悉，杭州市的小客车指标有摇号和竞拍两种配置方式，去年，全年杭州市个人摇号指标达58005个，此次新增的2万个指标占比大约为34.5%。而杭州去年个人竞价的指标数量为14300个。近日，浙江省人民政府办公厅发布《关于提振消费促进经济稳定增长的实施意见》，其中一条是要释放城乡汽车消费潜力。上述意见指出了几个方向，第一条便是鼓励杭州有序放宽汽车限购措施，制定汽车以旧换新和下乡惠农政策，深挖农村汽车消费潜力。值得注意的是，在春节销售淡季叠加疫情影响，中国车市刚刚经历了艰难的开局。据乘联会数据显示，2月我国狭义乘用车市场产量和批发销量分别为21.5万辆、25.2万辆，同比骤减80.6%、78.5%，销量规模相当于2006年2月水平。1-2月累计批发销量196.9万辆，同比下降41%，乘联会也再次调低了全年的销量预测增长率到-8%。在此背景之下，自疫情爆发以来，国家层面已多次发文鼓励汽车消费。2月16日，《求是》杂志发布的中央重要文章中指出，要稳定居民消费，鼓励汽车限购地区适当增加汽车号牌配额。2月20日，商务部在线上新闻发布会中，也提出鼓励汽车限购地区适当增加汽车号牌配额，带动汽车消费。3月13日，国家发改委、商务部等23个部门向各省市联合发布《关于促进消费扩容提质加快形成强大国内市场的实施意见》，提出要促进汽车限购向引导使用政策转变，鼓励汽车限购地区适当增加汽车号牌限额。而在刺激车市的措施中，放开部分城市的汽车限购在不少业内人士看来是最为直接，同时也是风险相对较低的做法。乘联会秘书长崔东树就指出，如果限购城市被限制的50万